# Zero-Shot Evaluation of DeepSeek R1 Models for Clinical Conversation Summarization

## Background and Motivation

The DeepSeek family of models represents an interesting advancement in reasoning-specialized language models. While DeepSeek published evaluation results in [their paper](https://arxiv.org/pdf/2501.12948) (see Table 5), I wanted to understand specifically how the various Distilled R1 models compare to the full R1 model on a practical use case: clinical conversation summarization using the ACI-Bench dataset.

This notebook demonstrates how to use **Lumigator** to systematically evaluate and compare these models. Lumigator provides a framework to:

1. Coordinate multiple model evaluations against the same dataset
2. Execute inference requests across different model deployments
3. Calculate standardized metrics for performance comparison
4. Organize and visualize the results for analysis

## Getting Started with Lumigator

To use this notebook, you'll need to have Lumigator running. In a terminal, run:

```bash
git clone git@github.com:mozilla-ai/lumigator.git
cd lumigator
make setup
echo $DEEPSEEK_API_KEY # This shouldn't be empty
echo $OPENAI_API_KEY # This shouldn't be empty, you need it for G-Eval metric
make start-lumigator-build

In [ ]:
from lumigator_sdk.lumigator import LumigatorClient

# Time to connect up to the Lumigator client!
LUMI_HOST = "localhost:8000"
client = LumigatorClient(api_host=LUMI_HOST)
print(f"Connection is: {client.health.healthcheck().status}")

## Dataset: ACI-Bench for Clinical Documentation

This evaluation uses the ACI-Bench dataset, which was introduced in the paper 
["ACI-Bench: a Novel Benchmark for Ambient Clinical Intelligence"](https://www.nature.com/articles/s41597-023-02487-3) 
(Yim et al., 2023). 

ACI-Bench was specifically designed to evaluate AI systems on their ability to 
understand doctor-patient conversations and generate accurate clinical documentation.

### About the Dataset

The test split of ACI-Bench that we'll be using consists of 40 doctor-patient conversations. 
These conversations aren't from real patient encounters but were created through professional medical simulations 
with standardized patients (actors trained to portray patients) and licensed physicians. 

This approach attempts to keep the data reasonably realistic while also being HIPAA-compliant, 
as no actual protected health information is included.

Each conversation includes:

1. A full transcript of the simulated clinical encounter, with speaker identification
2. Human-written reference documentation
3. Various sections of the standard clinical note format (SOAP - Subjective, Objective, Assessment, Plan)

### The Assessment & Plan Task

In this evaluation, we're specifically working with the **assessment and plan section** (`clef_taskC_test3_assessment_and_plan.json`), which is particularly challenging as it requires:

- Identifying the patient's medical conditions
- Understanding the physician's diagnostic reasoning
- Summarizing the recommended treatment approach
- Capturing follow-up plans and contingencies

This section of clinical documentation represents higher-level medical reasoning compared to other sections, making it a interesting test of a model's capacity for complex medical summarization and inference.

Each example in our dataset contains:
- `examples`: The full doctor-patient conversation transcript (with speaker turns marked as `[doctor]` and `[patient]`)
- `ground_truth`: The human-written assessment and plan section that serves as the reference summary
- `id`: A unique identifier for each conversation

The Assessment & Plan task was featured in the 2023 MEDIQA-CHAT shared task at CLEF (Conference and Labs of the Evaluation Forum)

### Limitations of This Evaluation

This evaluation has several important limitations that should be considered when interpreting the results:

1. **Unknown Training Data Exposure**: We cannot verify whether DeepSeek models were trained on the ACI-Bench dataset or similar clinical conversations. If any of these models were exposed to this data during training, they would have an unfair advantage in this evaluation - essentially having already "seen the answers" to the test. Without model cards or detailed training information disclosing training datasets, this remains an unknown factor.

2. **Relative Comparison Focus**: Given this limitation, our analysis primarily focuses on the relative performance differences between models within the DeepSeek family, rather than making absolute claims about their capabilities for clinical summarization. By comparing models from the same family, we can still draw meaningful conclusions about how performance scales with model size and architecture (Llama vs. Qwen) when all models would have had the same potential exposure to training data.

3. **Single Task Evaluation**: This evaluation examines performance on just one specific clinical documentation task (Assessment & Plan generation) and may not generalize to other medical tasks or to clinical summarization in different specialties or contexts.

4. **Simulated Data**: While the ACI-Bench dataset uses realistic simulated conversations, model performance might differ on real-world clinical conversations, which tend to be messier, less structured, and potentially contain more specialized terminology.

5. **Zero-Shot Setting**: Our evaluation uses a zero-shot approach with a specialized system prompt. 
Performance might improve significantly with few-shot examples.




In [ ]:
from pathlib import Path

import pandas as pd
import requests

# GitHub API URL to fetch the file list
download_url = "https://raw.githubusercontent.com/wyim/aci-bench/main/data/challenge_data_json/clef_taskC_test3_assessment_and_plan.json"
file_name = download_url.split("/")[-1]
save_dir = Path("data")
file_path = save_dir / file_name
save_dir.mkdir(parents=True, exist_ok=True)
response = requests.get(download_url)

data = response.json()
# convert it to a dataframe. The file by default has the columns 'src' and 'tgt'
df = pd.DataFrame(data["data"])  # noqa: PD901
# Rename the columns to "examples" and "ground_truth", which is what the Lumigator API expects for the data
df = df.rename(columns={"src": "examples", "tgt": "ground_truth", "file": "id"})  # noqa: PD901

processed_file_path = file_path.with_suffix(".csv")
# save it as a csv
df.to_csv(processed_file_path, index=False)

Great! Now the data is all formatted: let's take a look at an example to get a feel for what the data looks like. 
Understanding the data is crucial for interpreting the results and behavior of the models being evaluated. 

Every dataset
has quirks and unique things about it: in this notebook we won't dive too deeply into investigating the characteristics of the dataset,
but it's definitely worth taking more time to understand exactly what is in a dataset before you use it for anything.

In [ ]:
sample = df.iloc[0]
print("--- Snippet of Conversation ---")
print("\n".join(sample["examples"].split("\n")[6:8]))
print(" --- Assessment & Plan---")
print(sample["ground_truth"])

### Upload Dataset into Lumigator
Now, let's upload the dataset into lumigator using the Lumigator SDK. creating the dataset returns the dataset ID, which we will attach to future requests so that Lumigator knows which dataset should be used for running an eval.

In [ ]:
from pathlib import Path

from lumigator_schemas.datasets import DatasetFormat

# Upload that file that we created earlier
with Path.open(Path(processed_file_path), "r") as file:
    data = file.read()
dataset_response = client.datasets.create_dataset(dataset=data, format=DatasetFormat.JOB)
dataset_id = dataset_response.id
print(f"Dataset uploaded and has ID: {dataset_id}")

## Creating an Evaluation Pipeline in Lumigator

Now that we've uploaded our dataset, we'll create an experiment in Lumigator. In Lumigator terminology:

1. **Experiment** - A container that organizes related evaluation workflows
2. **Workflow** - A specific model configuration being evaluated against the dataset
3. **Dataset** - The collection of examples (in our case, clinical conversations)

This structure allows us to compare multiple models on the same dataset in a systematic way, with all results organized within a single experiment.

In [ ]:
# Now time to create an experiment in Lumigator! This is a container for all the workflows we'll run
from lumigator_schemas.experiments import ExperimentCreate

experiment_id = input("Enter the experiment ID, or press enter to create a new experiment: ")
if not experiment_id:
    request = ExperimentCreate(
        name="ACI-Bench clef_taskC_test3_assessment_and_plan",
        description="https://github.com/wyim/aci-bench/tree/main",
        dataset=dataset_id,
    )
    experiment_response = client.experiments.create_experiment(request)
    experiment_id = experiment_response.id
    print(f"Experiment created and has ID: {experiment_id}")

## Model Selection Rationale

For this evaluation, we're testing a range of DeepSeek models to understand how performance scales with model size and architecture:

- **DeepSeek R1** - The original reasoning-specialized model
- **DeepSeek-R1-Distill-Llama** variants (8B and 70B) - Knowledge distilled into Llama architecture
- **DeepSeek-R1-Distill-Qwen** variants (1.5B to 32B) - Knowledge distilled into Qwen architecture

This selection allows us to analyze:
1. How model size affects clinical summarization quality
2. Whether the base architecture (Llama vs Qwen) impacts performance
3. What performance tradeoffs come with using smaller distilled models

The smaller distilled models could be particularly valuable in resource-constrained clinical settings if they maintain adequate performance.

# Deploying Models for the DeepSeek Evaluation

To fully execute this notebook, you'll need to deploy the DeepSeek models yourself so that Lumigator can access them:

1. **Set up model deployments** for the DeepSeek models (both Llama and Qwen variants)
2. **Configure your `.env` file** with the IP addresses of your deployed models:
   ```
   # Llama models
   LLAMA_8B_IP=<your-deployment-ip>
   LLAMA_70B_IP=<your-deployment-ip>
   
   # Qwen models
   QWEN_1_5B_IP=<your-deployment-ip>
   QWEN_7B_IP=<your-deployment-ip>
   QWEN_14B_IP=<your-deployment-ip>
   QWEN_32B_IP=<your-deployment-ip>
   ```

For detailed instructions on how to deploy DeepSeek models on Kubernetes, see the guide on the Mozilla.ai blog: [Deploying DeepSeek V3 on Kubernetes](https://blog.mozilla.ai/deploying-deepseek-v3-on-kubernetes/).

In [ ]:
# These are all the models we want to evaluate
import os

from dotenv import load_dotenv
from utils import create_evaluation_config

# Load environment variables from .env file
load_dotenv()

evaluations = [
    # Note that you need to have run Lumigator with the DEEPSEEK_API_KEY environment variable set,
    # so that the Lumigator server can access the DeepSeek API
    {
        "name": "DeepSeek R1",
        "description": "DeepSeek R1 https://api-docs.deepseek.com/quick_start/pricing",
        "model": "deepseek-reasoner",
        "provider": "deepseek",
    },
    # vLLM deployments - Llama models
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Llama-8B", ip_address=os.getenv("LLAMA_8B_IP")),
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Llama-70B", ip_address=os.getenv("LLAMA_70B_IP")),
    # # vLLM deployments - Qwen models
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Qwen-7B", ip_address=os.getenv("QWEN_7B_IP")),
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Qwen-14B", ip_address=os.getenv("QWEN_14B_IP")),
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Qwen-32B", ip_address=os.getenv("QWEN_32B_IP")),
    create_evaluation_config(model_name="DeepSeek-R1-Distill-Qwen-1.5B", ip_address=os.getenv("QWEN_1_5B_IP")),
]

# Importance of the Custom System Prompt for Clinical Summarization

The custom system prompt is critical to the clinical conversation summarization task for several reasons:

1. **Domain-specific guidance**: By specifying that the model should act as an "expert medical scribe," we establish the specialized knowledge domain and expected level of expertise.

2. **Task definition**: The prompt clearly defines the task of converting conversational medical dialogue into a structured Assessment & Plan (A&P) document, which requires significant information distillation and reorganization.

3. **Format standardization**: The instruction to create "problem oriented" summaries in "narrative paragraph form" ensures consistent outputs across all model evaluations, making comparisons more meaningful.

4. **Clinical comprehensiveness**: By explicitly requesting information about "medical treatment, patient consent, patient education and counseling, and medical reasoning," the prompt ensures the models capture all critical components of medical documentation.

5. **Zero-shot performance**: Without this prompt, models would lack the context necessary to produce clinically useful summaries, especially the smaller distilled models being evaluated.

6. **Bias reduction**: The consistent prompt reduces variability in how different models interpret the task, allowing for more direct comparison of their inherent capabilities in medical summarization.

This prompt essentially serves as a controlled variable in our experiment, allowing us to focus on how different DeepSeek model variants perform on the same well-defined clinical task.

In [ ]:
system_prompt = """
You are an expert medical scribe who is tasked with reading the transcript of a conversation between a doctor and a patient,
and generating a concise Assessment & Plan (A&P) summary. 
Please follow the best standards and practices for modern scribe documentation.
The A&P should be problem oriented, with the assessment being a short narrative and the plan being a list with nested bullets.
When appropriate, please include information about medical treatment, patient consent, patient education and counseling, and medical reasoning.
""".strip()  # noqa: E501

In [ ]:
from lumigator_sdk.strict_schemas import WorkflowCreateRequest

# Configure generation parameters to ensure deterministic, high-quality outputs
# - temperature=0.0: Makes output deterministic (no randomness)
# - top_p=0.9: Limits token selection to the most probable ones
# - max_new_tokens=1024: Caps response length appropriately for reasoning + clinical summaries
# - frequency_penalty=0.0: No penalty for token repetition
generation_config = {
    "temperature": 0.0,
    "top_p": 0.9,
    "max_new_tokens": 1024,
    "frequency_penalty": 0.0,
}

# Create a workflow for each model configuration in our evaluation list
# Each workflow represents a single model's inference evaluation against the dataset
# within the experiment, allowing for systematic comparison of results
for evaluation_config in evaluations:
    # check if an evaluation by that name already exists
    existing_workflows = client.experiments.get_experiment(experiment_id).workflows
    existing_workflow = next(
        (workflow for workflow in existing_workflows if workflow.name == evaluation_config["name"]), None
    )
    if existing_workflow:
        # if status is failed, delete it
        if existing_workflow.status == "failed":
            client.workflows.delete_workflow(existing_workflow.id)
            print(f"Deleted failed workflow {evaluation_config['name']} with ID {existing_workflow.id}")
        else:
            print(f"Workflow {evaluation_config['name']} already exists with ID {existing_workflow.id}")
            continue
    request = WorkflowCreateRequest(
        name=evaluation_config["name"],
        description=evaluation_config["description"],
        model=evaluation_config["model"],
        provider=evaluation_config["provider"],
        base_url=evaluation_config.get("base_url"),
        dataset=dataset_id,
        experiment_id=experiment_id,
        system_prompt=system_prompt,
        generation_config=generation_config,
    )
    created_workflow = client.workflows.create_workflow(request)
    print(f"Created workflow {created_workflow.name} with ID {created_workflow.id}")

### Llamafile Workflows

In addition to all the DeepSeek models that are running remotely in DeepSeek or our own vLLM deployment, 
let's also compare how local models run with Llamafile stack up! We'll try a few different ones, conveniently available for 
us at https://huggingface.co/collections/Bojun-Feng/deepseek-distilled-llamafiles-50b-67a471e269c04acf9aa0c79b.

The amazing thing about llamafile is how simple it is! It's build on top of Llama.cpp, and using it is as simple as
downloading the file, opening up a terminal, and running:

```bash
$ chmod +x <file_name>.llamafile
$ ./<file_name>.llamafile
```
and Voila, the LLM server is running locally! Because it's running locally, 
we need to run these workflows one at a time: you'll need to run the code cell below a few times for each Llamafile you want to evaluate.
The process will be:

1. run the llamfile you want to test in a terminal window
2. Edit the code cell below so that it reflects the model_name you are testing
3. Run the cell, wait for it to finish
4. Go back to the terminal window and send ctrl+c to kill the process

Repeat these steps for each llamafile you want to evaluate.

I'm going to evaluate a few different types of the Llama 8B model. https://huggingface.co/Bojun-Feng/DeepSeek-R1-Distill-Llama-8B-GGUF-llamafile

For explanation about what each of these different suffixes mean 
(they're about quantization of gguf files), see https://github.com/ggml-org/llama.cpp/discussions/2094

* DeepSeek-R1-Distill-Llama-8B-Q2_K.llamafile
* DeepSeek-R1-Distill-Llama-8B-Q2_K_L.llamafile
* DeepSeek-R1-Distill-Llama-8B-Q4_K_M.llamafile
* DeepSeek-R1-Distill-Llama-8B-Q5_K_M.llamafile
* DeepSeek-R1-Distill-Llama-8B-Q6_K.llamafile



In [ ]:
from lumigator_sdk.strict_schemas import WorkflowCreateRequest

# make an eval config for a local llamafile mode
model_name = "DeepSeek-R1-Distill-Llama-8B-Q8_0"
evaluation_config = create_evaluation_config(model_name=model_name, ip_address="localhost", port=8080)

# check if an evaluation by that name already exists
existing_workflows = client.experiments.get_experiment(experiment_id).workflows
existing_workflow = next(
    (workflow for workflow in existing_workflows if workflow.name == evaluation_config["name"]), None
)
if existing_workflow and existing_workflow.status == "failed":
    client.workflows.delete_workflow(existing_workflow.id)
    print(f"Deleted failed workflow {evaluation_config['name']} with ID {existing_workflow.id}")
    existing_workflow = None

if existing_workflow:
    print(f"Workflow {evaluation_config['name']} already exists with ID {existing_workflow.id}")
else:
    request = WorkflowCreateRequest(
        name=evaluation_config["name"],
        description=evaluation_config["description"],
        model=evaluation_config["model"],
        provider=evaluation_config["provider"],
        base_url=evaluation_config.get("base_url"),
        dataset=dataset_id,
        experiment_id=experiment_id,
        system_prompt=system_prompt,
        generation_config=generation_config,
        job_timeout_sec=60 * 60 * 2,
    )
    client.workflows.create_workflow(request).model_dump()

## Executing the Evaluation Workflows

With all workflows now created, Lumigator will:

1. Generate summaries from each model for every example in the dataset
2. Calculate performance metrics like ROUGE, BLEU, and BERTScore
3. Make all results available for comparison

This automated evaluation approach ensures consistent testing conditions across all models. The wait_for_all_workflows function will poll the Lumigator API until all workflows complete, allowing us to retrieve and analyze the results.

In [ ]:
import pandas as pd
from utils import compile_and_display_results, wait_for_all_workflows

print(f"Waiting for all workflows to complete for experiment {experiment_id}")
experiment = wait_for_all_workflows(client, experiment_id)
print("All workflows completed!")
workflow_details, styled_df = compile_and_display_results(client, experiment)
display(styled_df)

In [ ]:
# print the ground truth of an example
example = 1
print("Ground Truth of Example:")
print(df.iloc[example]["ground_truth"])
print("=" * 50)
# for each model print its name and prediction
for workflow in workflow_details:
    print(f"Model: {workflow}")
    print("==" * 50)
    print(workflow_details[workflow]["artifacts"]["reasoning"][example])
    print("=" * 50)
    print(workflow_details[workflow]["artifacts"]["predictions"][example])
    print("-" * 50)